# Task 1 wind fetch scratchpad

Quick notebook to pull DYAMOND GEOS wind components (U/V/W) for a single time/face/level, summarize them, and prep for streamlines later. Keep reads coarse (`quality` negative) so we don't overpull multi-TB data.

## What this does
- Uses the public NSDF `.idx` endpoints for GEOS wind (`u_face_{face}_depth_52_time_0_10269.idx` pattern).
- Fetches U, V, W at a chosen face/time with a downsample (`quality = -resolution_drop`).
- Prints shapes/ranges so we know the data is sane.
- (Optional) Saves small slices to disk if you uncomment the save block.

We still need to map cubed-sphere indices to lat/lon when exporting streamlines; that grid metadata isn't pulled here.

In [ ]:
import numpy as np
import pandas as pd
import OpenVisus as ov
from pathlib import Path

## Config
Adjust these before running.
- `IDX_TEMPLATE` uses `{var_upper}`, `{var_lower}`, `{face}` tokens.
- `time_index` is the timestep (0 = first frame).
- `resolution_drop` becomes a negative `quality` for coarser reads (safer).

In [ ]:
# DYAMOND GEOS cube-face URL pattern (U/V/W share the same pattern)
IDX_TEMPLATE = (
    "https://nsdf-climate3-origin.nationalresearchplatform.org:50098/"
    "nasa/nsdf/climate3/dyamond/GEOS/GEOS_{var_upper}/{var_lower}_face_{face}_depth_52_time_0_10269.idx"
)

# Run parameters
face_index = 0      # cubed-sphere face 0-5
time_index = 0      # timestep index (0 = first frame)
resolution_drop = 6 # higher = coarser (use 6-10 for quick tests)

output_dir = Path("data/raw/geos")  # optional save target; not created by default

## Helpers

In [ ]:
def make_idx_url(var: str, face: int) -> str:
    return IDX_TEMPLATE.format(var_upper=var.upper(), var_lower=var.lower(), face=face)


def load_field(var: str, face: int, time: int, res_drop: int):
    url = make_idx_url(var, face)
    ds = ov.LoadDataset(url)
    quality = -res_drop  # OpenVisus: negative = coarser, 0 = full
    arr = ds.read(field=var.lower(), time=time, quality=quality)
    return url, arr


def summarize(name: str, arr: np.ndarray) -> dict:
    return {
        "field": name,
        "shape": tuple(arr.shape),
        "min": float(arr.min()),
        "max": float(arr.max()),
        "mean": float(arr.mean()),
    }

## Fetch U/V/W at the current face/time
This uses a coarse read (`quality = -resolution_drop`). Adjust `resolution_drop` lower if you need more detail (at the cost of bandwidth/time).

In [ ]:
summaries = []
arrays = {}
for var in ["u", "v", "w"]:
    url, arr = load_field(var, face=face_index, time=time_index, res_drop=resolution_drop)
    arrays[var] = arr
    summaries.append({"url": url, **summarize(var, arr)})

pd.DataFrame(summaries)

## Optional: inspect a small slice
Pick a vertical level (k) and show a tiny patch for sanity. Adjust slicing to match your axis order.

In [ ]:
k = 0  # vertical level index
row_slice = slice(0, 5)
col_slice = slice(0, 5)

patch = arrays["u"][row_slice, col_slice, k]
print("U patch (5x5 at level k=0):")
print(pd.DataFrame(patch))

## Optional: save downsampled arrays
Uncomment to write `.npy` snapshots for later processing (kept small via downsampling).

In [ ]:
# output_dir.mkdir(parents=True, exist_ok=True)
# for var, arr in arrays.items():
#     target = output_dir / f"{var}_face{face_index}_t{time_index}_q{resolution_drop}.npy"
#     np.save(target, arr)
#     print("saved", target)


## Map cubed-sphere indices to lat/lon (grid metadata required)
To project streamlines onto the globe, you need lat/lon for each (i,j) on the cubed-sphere faces. The GEOS grid metadata should provide per-face lat/lon arrays. Point `GRID_TEMPLATE` to your lat/lon files (one per face) and load them here.

Expected: a file per face containing `lat` and `lon` arrays shaped like U/V/W (e.g., `(1440, 1440)` for the surface grid).


In [ ]:
from pathlib import Path

# Replace this with the actual path or URL for the face grid metadata (lat/lon arrays)
# Example pattern if you download per-face NPZ files locally:
GRID_TEMPLATE = Path("data/raw/geos/latlon_face_{face}.npz")


def load_face_latlon(face: int):
    grid_path = Path(str(GRID_TEMPLATE).format(face=face))
    if not grid_path.exists():
        print(f"Grid file not found: {grid_path}")
        return None, None
    data = np.load(grid_path)
    lat = data.get("lat")
    lon = data.get("lon")
    print(f"Loaded lat/lon for face {face}: lat shape {lat.shape}, lon shape {lon.shape}")
    return lat, lon

# Example: try to load current face
lat_face, lon_face = load_face_latlon(face_index)

if lat_face is not None and lon_face is not None:
    i, j = 0, 0
    print(f"Sample lat/lon at (i={i}, j={j}): lat={lat_face[i, j]}, lon={lon_face[i, j]}")


## Next steps
- Map cubed-sphere (i,j) indices to lat/lon using the GEOS face grid metadata.
- Integrate streamlines per face using U/V on the slice/level you care about; convert to lat/lon/alt.
- Export to `data/samples/streamlines.json` per the existing schema for the web client.